Quellen

https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

https://stackoverflow.com/questions/56030884/how-to-define-specific-number-of-convolutional-kernels-filters-in-pytorch

https://www.kaggle.com/hanjoonchoe/cnn-time-series-forecasting-with-pytorch

https://medium.com/@sumanshusamarora/understanding-pytorch-conv1d-shapes-for-text-classification-c1e1857f8533

https://datascience.stackexchange.com/questions/78030/multivariate-time-series-analysis-when-is-a-cnn-vs-lstm-appropriate

https://discuss.pytorch.org/t/solved-concatenate-time-distributed-cnn-with-lstm/15435

https://d2l.ai/chapter_convolutional-modern/resnet.html

http://jmir.sourceforge.net/manuals/jSymbolic_manual/featureexplanations_files/featureexplanations.html

https://medium.com/analytics-vidhya/understanding-and-implementation-of-residual-networks-resnets-b80f9a507b9c

https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm

In [ ]:
!pip install music21 

In [ ]:
import pandas as pd
# from music21 import converter, corpus, instrument, midi, note, chord, pitch, meter
from music21 import *
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import numpy as np

import os # handle files 
from torch.utils.data import Dataset 

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html
# L_out = (L_in+2*padding-dilation*(kernel_size-1)-1)*0.5 + 1
def whatPad(L_in,L_out,kernel_size,stride,dilation):
    pad = ((L_out-1)*stride-L_in+dilation*(kernel_size-1)+1)*0.5
    return pad

L_in = 128
L_out = L_in
kernel_size = 5
stride = 1
dilation = 1

print(whatPad(L_in,L_out,kernel_size,stride,dilation))

In [ ]:
# Angepasste Dataset-Klasse
# Liest ein Verzeichnis mit Komponisten-Unterverzeichnis mit Midi-Dateien ein.
# Aus den Midi-Datein werden pro 'Part' 'num_subparts' viele Teile mit 'num_notes' zusammenhängenden Noten extrahiert.
# Aus den Noten-Objekten werden Features ausgelesen (z.B. Midi-Zahl und Quarter Length).
# Damit entsteht für jeden Subpart eine verschachtelte Liste, für die gilt Länge = Anzahl Noten = n.
# Für jeden Note gibt es in dieser Liste eine eigene Liste mit den zur Note gehörigen Features.
# --> [[Midi-Zahl_0,Quarter_Length_0],[Midi-Zahl_1,Quarter_Length_1],...,[Midi-Zahl_n-1,Quarter_Length_n-1]]
# oder für m = Anzahl Feature (pro Note)
# --> [[Feature_0_0,Feature_0_1,...,Feature_0_m-1],...,[Feature_n-1_0,Feature_n-1_1,...,Feature_n-1_m-1]]
# Erstellt Torch-Tensoren aus den Listen, deren 'shape' angepasst ist um den conv1d Layern als Input gefüttert zu werden.
class FeatureDataset(Dataset):
    
    def __init__(self, path, composer_list, num_notes, num_subparts, num_features):
        
        # Eingaben abspeichern, falls man bei einem bereits initialisierten FeatureDataset-Objekt nochmal darauf zugreifen möchte
        self.path = path
        self.composer_list = composer_list
        self.num_subparts = num_subparts
        self.num_notes = num_notes
        self.num_features = num_features
        
        # Leere Listen für das Extrahieren der Daten
        subparts_notes = [] # wir extrahieren zu erst die subparts als Liste von Note-Objekten
        x_train = [] # hier werden später die einzelnen subparts gesammelt
        y_train = [] # hier werden die zu den subparts gehörenden Komponisten IDs gesammelt
        
        # Datei einlesen
        composer_id = 0
        for composer in self.composer_list: # alle Komponisten durchgehen
            
            print(composer)
            print('----------*----------*----------')
            
            directory = os.path.join(self.path,composer) # bestimme das Verzeichnis für den Komponisten. Urpsrungspfad mit dem Komponistennamen zusammenführen -> Unterverzeichnis
            for file in os.listdir(directory): # alle Dateien im Verzeichnis des Komponisten durchgehen
                filename = os.fsdecode(file) # Dateiname als String speichern, zum vergleichen in der nächsten Zeile
                if filename.endswith(".mid"): # wenn die Datei eine Midi-Datei ist, weiterverarbeiten, sonst ignorieren
                    
                    print(filename)
                    
                    path_to_midi = os.path.join(directory, filename) # Pfad zur Datei erstellen (s.o.)
                    score = converter.parse(path_to_midi) # mit dem Music21 converter die Datei auslesen und in ein Score-Objekt umwandeln
                    
                    print(len(score.parts),' parts')
                    print('----------*----------')
                    
                    # for i,part in enumerate(score.parts): # alle parts im Score Objekt durchegehen
                    for part in score.parts: # Score Objekte bestehen eventuell aus mehreren Part Objekte. Daher alle Parts durchgehen.
                        for _ in range(self.num_subparts): # num_subparts oft getNNotes aufrufen
                            notes = self.getNNotes(part, self.num_notes)
                            if len(notes)>0: # falls getNNotes keine leere Liste zurückgegeben hat:
                                subparts_notes.append(notes) # in subparts_notes ablegen
                                y_train.append(composer_id) # den Komponisten in y ablegen
                                
            composer_id += 1 # nächster Komponist
        
        
        # Bis jetzt haben wir nur Noten-Objekte ausgelesen
        
        # Features aus den Noten-Objekten auslesen
        for subpart in subparts_notes: # die einzelnen Subparts wieder durchlaufen
            subpart_list = [] # leere Liste für jeden Subpart erstellen
            for note in subpart: # die Noten im Subpart durchlaufen
                feature_list = [] 
                feature_list.append(note.pitch.ps/127) # MIDI-Zahl, normieren auf [0,1]
                feature_list.append(note.quarterLength) # Dauer in Viertel-Länge
                subpart_list.append(feature_list.copy()) # Kopie(!) der einzelnen Features ablegen
            x_train.append(subpart_list.copy())
            
        
        # Erstellen von Tensoren.
        # Das Torch-Netzwerk möchte mit float32 Werten arbeiten
        # torch.nn.CrossEntropyLoss() braucht die Y-Werte mit dtype = long
        
        self.X_train = torch.tensor(x_train, dtype=torch.float32) #float32
        self.Y_train = torch.tensor(y_train, dtype=torch.long)
        
        print("Training Shape before reshape", self.X_train.shape, self.Y_train.shape)
        
        # Conv1d Layer benötigen die Input-Werte in dieser bestimmten Form [Batch_Size, Input_Channel, Sequence_Length]
        # für LSTM oder auch Linear (bzw. Dense), kann dies wieder anders aussehen.
        
        self.X_train = torch.reshape(self.X_train, (self.X_train.shape[0], self.num_features, self.X_train.shape[1]))
        
        print("Training Shape after reshape", self.X_train.shape, self.Y_train.shape)
    
    # vermutlich für den Dataloader nötig
    def __len__(self):
        return len(self.Y_train)
    
    # get-Methode muss entsprechend der Tensoren angepasst werden, ist hier recht simpel
    def __getitem__(self, idx):
        return self.X_train[idx], self.Y_train[idx]
    
    # n zusammenhängende Noten aus einem Part (zufälliger n-Subpart)
    # Dazu wird ein zufälliger, aber zulässiger Startindex im Part ausgewählt, ab diesem werden dann die Note-Objekte ausgelesen
    def getNNotes(self, part, n):  
        notes_list = [] # leere Liste um die Note-Onjekte aufzunehmen
        flat_part = part.flat # Auflösen der verschachtelten Struktur das Part-Objektes -> Flatten = 'platt machen'
        noteIter= flat_part.getElementsByClass(note.Note) # Iterator über alle Note-Objekte im aufgelösten Part
    
        num_notes = len(noteIter) # damit hat die n-te Note den Index num_notes-1

        '''
        #Ist in der nächsten IF-Abfrage enthalten
        if numNotes < n: # der Part muss mindestens n Noten enthalten, sonst können wir keinen n-langen Subpart bilden
            return notesList # -> Rückgabe von leerer Liste
        '''
        
        max_start_index = num_notes-1-n # maximaler Index an dem man anfangen kann um einen n-langen Subpart zu bilden
        
        if max_start_index < 0: # Ist der berechnete Index negativ, sind weniger als n Noten im Part enthalten
            return notes_list # -> Rückgabe von leerer Liste
        
        
        start_index = random.randint(0,max_start_index) # wir können einen zufälligen Index zwischen 0 und dem maximal möglichen Startindex wählen

        for i in range(n): # n Noten ab dem Startindex in der Liste ablegen
            notes_list.append(noteIter[start_index+i])

        return notes_list

# Modell

In [ ]:
# Da eindimensionale Zeitreihendaten (wie oben beschrieben) vorliegen, genügen Conv1D-Layer, statt der z.B. aus der Bildverarbeitung bekannten Conv2D-Layer.
# Die Hyperparameter sind zum Großteil durch Ausprobieren entstanden.
class CONV_LSTM1(nn.Module):
    def __init__(self, num_classes, hidden_size, batch_s, num_features, num_filter1, num_filter2, num_fitler3):
        super(CONV_LSTM1, self).__init__()
        
        self.debug = False # Falls True, printet der Forward Pass die Gestalten der Tensoren.
        self.res = True # Falls Ture, wird die 'Abkürzung' für den Residualblock genutzt. Falls False, ist das Netzwerk eine normales Conv-Net.
        
        # ----------*----------*----------
        # Allgemeine Parameter
        
        self.batch_size = batch_s # batch size
        self.num_classes = num_classes # Anzahl der Klassen (Komponisten), relevant für den letzten Layer
        
        # Conv Parameter
        self.num_features = num_features # Anzahl der Inputchannels des ersten Conv-Layers
        self.num_filter1 = num_filter1
        self.num_filter2 = num_filter2
        self.num_filter3 = num_filter3
        
        # LSTM Paramter
        self.hidden_size = hidden_size # hidden state
        self.num_layers = 1 # number of layers
        
        # ----------*----------*----------*----------*----------*----------*----------*----------*----------
        # Layer
        # ---------- Res Block 1 Start ----------
        self.conv1_id = nn.Conv1d(self.num_features, out_channels=self.num_filter1, kernel_size = 1, stride = 1)
        self.b_norm1_id = nn.BatchNorm1d(self.num_filter1)
        
        self.conv1a = nn.Conv1d(self.num_features, out_channels=self.num_filter1, kernel_size=5, padding = 2)
        self.b_norm1a = nn.BatchNorm1d(self.num_filter1)
        
        self.conv1b = nn.Conv1d(self.num_filter1, out_channels=self.num_filter1, kernel_size=5, padding = 2)
        self.b_norm1b = nn.BatchNorm1d(self.num_filter1)
        
        self.conv1c = nn.Conv1d(self.num_filter1, out_channels=self.num_filter1, kernel_size=5, padding = 2)
        self.b_norm1c = nn.BatchNorm1d(self.num_filter1)
        
        # ---------- Res Block 1 Ende ----------
        
        self.drop1 = nn.Dropout(p=0.1) # Standard Dropout Layer für stabileres Training
        
        self.pool1 = nn.MaxPool1d(2, stride = 2) # Standard Pooling, auch hier 1-D
        
        # ---------- Res Block 2 Start ----------
        self.conv2_id = nn.Conv1d(self.num_filter1, out_channels=self.num_filter2, kernel_size = 1, stride = 1)
        self.b_norm2_id = nn.BatchNorm1d(self.num_filter2)
        
        self.conv2a = nn.Conv1d(self.num_filter1, out_channels=self.num_filter2, kernel_size=5, padding = 2)
        self.b_norm2a = nn.BatchNorm1d(self.num_filter2)
        
        self.conv2b = nn.Conv1d(self.num_filter2, out_channels=self.num_filter2, kernel_size=5, padding = 2)
        self.b_norm2b = nn.BatchNorm1d(self.num_filter2)
        
        self.conv2c = nn.Conv1d(self.num_filter2, out_channels=self.num_filter2, kernel_size=5, padding = 2)
        self.b_norm2c = nn.BatchNorm1d(self.num_filter2)
        
        # ---------- Res Block 2 Ende ----------
        
        self.drop2 = nn.Dropout(p=0.25) # Standard Dropout Layer für stabileres Training
        
        self.pool2 = nn.MaxPool1d(2, stride = 2) # Standard Pooling, auch hier 1-D
        
        # ---------- Res Block 3 Start ----------
        self.conv3_id = nn.Conv1d(self.num_filter2, out_channels=self.num_filter3, kernel_size = 1, stride = 1)
        self.b_norm3_id = nn.BatchNorm1d(self.num_filter3)
        
        self.conv3a = nn.Conv1d(self.num_filter2, out_channels=self.num_filter3, kernel_size=5, padding = 2)
        self.b_norm3a = nn.BatchNorm1d(self.num_filter3)
        
        self.conv3b = nn.Conv1d(self.num_filter3, out_channels=self.num_filter3, kernel_size=5, padding = 2)
        self.b_norm3b = nn.BatchNorm1d(self.num_filter3)
        
        self.conv3c = nn.Conv1d(self.num_filter3, out_channels=self.num_filter3, kernel_size=5, padding = 2)
        self.b_norm3c = nn.BatchNorm1d(self.num_filter3)
        
        # ---------- Res Block 3 Ende ----------
        
        self.drop3 = nn.Dropout(p=0.5) # Standard Dropout Layer für stabileres Training
        
        self.pool3 = nn.MaxPool1d(2, stride = 2) # Standard Pooling, auch hier 1-D
        
        # Die convolotional Layer sollen idealerweise kurzfristige Zusammenhänge/Muster aus der Zeitreihe herausarbeiten
        # Für langfristige Strukturen nutzt man zusätzlich einen LSTM-Layer.
        self.lstm1 = nn.LSTM(input_size=self.num_filter3, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True) 
        
        self.flat1 = nn.Flatten()
        
        self.fc1 = nn.Linear(4096,1024)
        
        self.fc2 = nn.Linear(1024,512)
        
        self.fc3 = nn.Linear(512, self.num_classes)
        # ----------*----------*----------*----------*----------*----------*----------*----------*----------
        
        
    def forward(self,x):
        if self.debug:
            print('----------*--------- Forward Start ----------*----------')
            print('Input Shape')
            print(x.shape)
            print('----------Res Block 1 Start--------')
                
        # ----------*----------*----------*----------*----------*----------
        # Res Block 1
        # ----------*----------*----------
        # Layer 1_id - Conv
        if self.res:
            if self.debug:
                print('conv1_id')
            x_id = self.conv1_id(x)
            if self.debug:
                print(x_id.shape)
        
        # ----------*----------*----------
        # Layer 1a - Conv Layer
        if self.debug:
            print('conv1a')
        x_res = self.conv1a(x) 
        x_res = self.b_norm1a(x_res)
        x_res = F.relu(x_res) 
        
        # ----------*----------*----------
        # Layer 1b - Conv Layer
        if self.debug:
            print(x_res.shape)
            print('conv1b')
            
        x_res = self.conv1b(x_res)
        x_res = self.b_norm1b(x_res)
        x_res = F.relu(x_res) 
            
        # ----------*----------*----------
        # Layer 1c - Conv Layer
        if self.debug:
            print(x_res.shape)
            print('conv1c')
            
        x_res = self.conv1c(x_res)
        x_res = self.b_norm1c(x_res)
        x_res = F.relu(x_res)
        
        if self.debug:
            print(x_res.shape)
            print('----------Res Block 1 Ende--------')
        # ----------*----------*----------  
        # Aktivierung 
        if self.res:
            if self.debug:
                print('Relu(x+x_res)')
            x = x_id+x_res
        else:
            if self.debug:
                print('Relu(x)')
            x = x_res
        x = F.relu(x)
        
        # Dropout 
        if self.debug:
            print(x.shape)
            print('drop1')
        x = self.drop1(x)
        
        
        # Pool
        if self.debug:
            print(x.shape)
            print('pool1')
        x = self.pool1(x)
        
        
        # ----------*----------*----------*----------*----------*----------
        # Res Block 2
        if self.debug:
            print(x.shape)
            print('----------Res Block 2 Start--------')       
        # ----------*----------*----------
        # Layer 2_ind - Conv Layer
        if self.res:
            if self.debug:
                print('conv2_id')
            x_id = self.conv2_id(x)
            if self.debug:
                print(x_id.shape)
        # ----------*----------*----------
        # Layer 2a - Conv Layer
        if self.debug:
            print('conv2a')
            
        x_res = self.conv2a(x) 
        x_res = self.b_norm2a(x_res)
        x_res = F.relu(x_res)  
        # ----------*----------*----------
        # Layer 2b - Conv Layer
        if self.debug:
            print(x_res.shape)
            print('conv2b')
            
        x_res = self.conv2b(x_res)
        x_res = self.b_norm2b(x_res)
        x_res = F.relu(x_res) 
        
        if self.debug:
            print(x_res.shape)
        
        # ----------*----------*----------
        # Layer 2c - Conv Layer
        if self.debug:
            print(x_res.shape)
            print('conv2c')
            
        x_res = self.conv2c(x_res)
        x_res = self.b_norm2c(x_res)
        x_res = F.relu(x_res)
        
        if self.debug:
            print(x_res.shape)
            print('----------Res Block 2 Ende--------')
        # ----------*----------*----------  
        # Aktivierung 
        if self.res:
            if self.debug:
                print('Relu(x+x_res)')
            x = x_id+x_res
        else:
            if self.debug:
                print('Relu(x)')
            x = x_res
        x = F.relu(x)
        
        # Dropout 
        if self.debug:
            print(x.shape)
            print('drop2')
        x = self.drop2(x)    
        
        # Pool
        if self.debug:
            print(x.shape)
            print('pool2')
        x = self.pool2(x)
        
        # ----------*----------*----------*----------*----------*----------
        # Res Block 3
        if self.debug:
            print(x.shape)
            print('----------Res Block 3 Start--------')       
        # ----------*----------*----------
        # Layer 2_ind - Conv Layer
        if self.res:
            if self.debug:
                print('conv3_id')
            x_id = self.conv3_id(x)
            if self.debug:
                print(x_id.shape)
        # ----------*----------*----------
        # Layer 3a - Conv Layer
        if self.debug:
            print('conv3a')
            
        x_res = self.conv3a(x) 
        x_res = self.b_norm3a(x_res)
        x_res = F.relu(x_res)  
        # ----------*----------*----------
        # Layer 3b - Conv Layer
        if self.debug:
            print(x_res.shape)
            print('conv3b')
            
        x_res = self.conv3b(x_res)
        x_res = self.b_norm3b(x_res)
        x_res = F.relu(x_res) 
        
        if self.debug:
            print(x_res.shape)
        
        # ----------*----------*----------
        # Layer 3c - Conv Layer
        if self.debug:
            print(x_res.shape)
            print('conv3c')
            
        x_res = self.conv3c(x_res)
        x_res = self.b_norm3c(x_res)
        x_res = F.relu(x_res)
        
        if self.debug:
            print(x_res.shape)
            print('----------Res Block 3 Ende--------')
        # ----------*----------*----------  
        # Aktivierung 
        if self.res:
            if self.debug:
                print('Relu(x+x_res)')
            x = x_id+x_res
        else:
            if self.debug:
                print('Relu(x)')
            x = x_res
        x = F.relu(x)
        
        # Dropout 
        if self.debug:
            print(x.shape)
            print('drop3')
        x = self.drop3(x)    
        
        # Pool
        if self.debug:
            print(x.shape)
            print('pool3')
        x = self.pool3(x)
        
        # ----------*----------*----------*----------*----------*----------

        if self.debug:
            print(x.shape)
            print('permute')
        
        # ----------*----------*----------
        # Layer 6 - LSTM Layer
        '''
        x = x.permute(0,2,1) # wegen Batch_First = True im LSTM Layer 
        
        if self.debug:
            print(x.shape)
            print('lstm')
        
        h_0_1 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state - 'cell state'
        c_0_1 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state - 'output'
        x, (hn, cn) = self.lstm1(x, (h_0_1, c_0_1)) # dem LSTM muss Input, Hidden und Internal State übergeben werden
        '''
        # ----------*----------*----------
        # Layer 7 - Flat Layer
        if self.debug:
            print(x.shape)
            print('flat')
        
        x = self.flat1(x)
        
        # ----------*----------*----------
        # Layer 8 - Dense Layer
        if self.debug:
            print(x.shape)
            print('fc1')
  
        x = self.fc1(x)
        x = F.relu(x) 
        
        # ----------*----------*----------
        # Layer 9 - Dense Layer
        if self.debug:
            print(x.shape)
            print('fc2')
            
        x = self.fc2(x)
        x = F.relu(x)
        
        # ----------*----------*----------
        # Layer 10 - Dense Layer Output
        if self.debug:
            print(x.shape)
            print('fc3/output')
        
        x = self.fc3(x)
        
        if self.debug:
            print(x.shape)
            print('----------*--------- Forward Ende ----------*----------')
        
        return x

# Hyperparamter für den Datensatz

In [ ]:
# Datensatz einlesen und in ein Dataset übertragen
path = '../input/musicnet-dataset/musicnet_midis/musicnet_midis/'
path_train = '../input/musicnet-midis-testsplit/musicnet_midis/Training'
path_test = '../input/musicnet-midis-testsplit/musicnet_midis/Test'

composerList = ['Brahms','Dvorak','Cambini','Faure','Haydn','Ravel']
#composerList = ['Brahms','Ravel']

n_Notes = 128
n_Subparts = 25
n_features = 2

# Batchsize
batch_s = 10

In [ ]:
os.listdir('../input/musicnet-midis-testsplit/musicnet_midis/Training')
os.listdir(path_test)

## Erzeugen des Feature-Set-Objekts und des Trainings- Und Validierungs-Loaders

In [ ]:
feature_set = FeatureDataset(path_train, composerList, n_Notes, n_Subparts, n_features)
test_set = FeatureDataset(path_test, composerList, n_Notes, n_Subparts, n_features)

In [ ]:
# Split
train_size = int(0.8 * len(feature_set)) # split 0.2
val_size = len(feature_set) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(feature_set, [train_size, val_size])

# train_loader erstellen
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_s, shuffle = True)
validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_s)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_s)

# Hyperparamter für das Modell

In [ ]:
num_classes = len(composerList)
input_size = n_Notes 
#hidden_size = int(input_size/5)
hidden_size = 64
#num_features = 4
learning_rate = 0.0001 

num_filter1 = 64
num_filter2 = 128
num_filter3 = 256

In [ ]:
conv_lstm1 = CONV_LSTM1(num_classes, hidden_size, batch_s, n_features,num_filter1,num_filter2,num_filter3)

# Loss und Optimizer festlegen, beide standard bei Multiclass-Klassifizierung
# torch.nn.CrossEntropyLoss ist nicht = categorical cross entropy aus Tensorflow
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(conv_lstm1.parameters(), lr=learning_rate) 

# Das untrainierte Modell

In [ ]:
# Zur Bewertung der Performance 
def hit(target, pred): # nimmt Zielwerte und Vorhersagen an und vergleicht diese
    # das LSTM gibt keine als Wahrscheinlichkeitsverteilung interpretierbaren Werte aus, deshalb muss Softmax angewandt werden.
    # Dies ist eine Art Pytorch Eigenheit, man könnte auch im Modell auf den letzten Layer Softmax anwenden, dann funktioniert 
    # torch.nn.CrossEntropyLoss() aber nicht mehr.
    
    #pred_softmax = torch.softmax(pred, dim = 1)
    
    #print('----------*----------*----------')
    pred_softmax = torch.log_softmax(pred.view(-1),0)
    
    #print(pred_softmax)
    
    # der folgenden Befehl könnte sicher auch einfach gestaltet werden
    # detach() - entfernt Gradienten, welche einem Tensor zugewiesen werden
    # reshape(-1) - plättet (flatten) einen Tensor, d.h. wirft alle Werte in einer Dimension zusammen
    # numpy() - wandelt den Tensor in ein Numpy Array um
    # argmax() - gibt den Index mit dem maximalen Wert zurück
    # -> argmax gibt für die 1-Hot-Kodierung die Klasse zurück,
    # -> argmax gibt für einen Vektor mit Klassen-Wahrscheinlichkeiten, die Klasse mit der maximalen Wahrscheinlichkeit zurück
    
    if target.detach().reshape(-1).numpy() == pred_softmax.detach().reshape(-1).numpy().argmax():
        return True
    else:
        return False

In [ ]:
conv_lstm1.debug = False
conv_lstm1.res = True

In [ ]:
def perf_val(): # performance auf val-set
    conv_lstm1.eval()
    hits = 0
    tries = 0
    debug = False
    for x_val, y_val in validation_loader:
        if debug:
            print('----------*----------')
            print('x_val')
            print(x_val.shape)
            print(x_val)
        
        yhat = conv_lstm1(x_val)
    
    
        for target, pred in zip(y_val, yhat):
            tries += 1
            if hit(target, pred):
                hits +=1
            
    print('Tries')
    print(tries)
    print('Number of hits:')
    print(hits)
    print('Ratio')
    print(hits/tries)
    # bei einem zufällig gewichteten/untrainierten Modell würde man eine Treffsicherheit von 1:(Anzahl Klassen) erwarten
    print('Random Ratio')
    print(1/num_classes)

In [ ]:
def perf_test():# performance auf dem test set
    conv_lstm1.eval()
    hits = 0
    tries = 0
    for x_val, y_val in test_loader:
        yhat = conv_lstm1(x_val)
        for target, pred in zip(y_val, yhat):
            tries += 1
            if hit(target, pred):
                hits +=1
            
    print('Tries')
    print(tries)
    print('Number of hits:')
    print(hits)
    print('Ratio')
    print(hits/tries)
    # bei einem zufällig gewichteten/untrainierten Modell würde man eine Treffsicherheit von 1:(Anzahl Klassen) erwarten
    print('Random Ratio')
    print(1/num_classes)

# Training

In [ ]:
num_epochs = 25

avg_losses = []
conv_lstm1.train()
debug = False
val_each = 1
test_each = 1
for epoch in range(num_epochs):
    conv_lstm1.train()
    epoch_losses = []
    for x_batch, y_batch in train_loader:
        output = conv_lstm1.forward(x_batch) #forward pass
        optimizer.zero_grad()#caluclate the gradient, manually setting to 0
        
        if debug:
            print(output.shape)
            print(y_batch.shape)
        #print(y_batch)
        #print(torch.max(y_batch,1))
        #loss = criterion(output.view(1,-1), y_batch)

        
        #output = torch.reshape(output, (output.shape[0], output.shape[2]))
        loss = criterion(output, y_batch)
        epoch_losses.append(loss.item())
        loss.backward() #calculates the loss of the loss function
         
        optimizer.step() #improve from loss, i.e backprop
        #losses.append(loss)
    
    epoch_avg = sum(epoch_losses)/len(epoch_losses)
    avg_losses.append(epoch_avg)
    print("Epoch: %d, avg loss: %1.5f" % (epoch+1, epoch_avg)) 
    if epoch % val_each == 0:
        print('----------Val---------')
        perf_val()
        
    if epoch % test_each == 0:
        print('---------Test--------')
        perf_test()
    
    print('---------*---------*----------*---------*---------*----------*---------*---------*----------')
    

# Nach dem Training

In [ ]:
# performance vor dem Training
conv_lstm1.eval()
hits = 0
tries = 0
for x_val, y_val in validation_loader:
    yhat = conv_lstm1(x_val)
    for target, pred in zip(y_val, yhat):
        tries += 1
        if hit(target, pred):
            hits +=1
            
print('Tries')
print(tries)
print('Number of hits:')
print(hits)
print('Ratio')
print(hits/tries)
# bei einem zufällig gewichteten/untrainierten Modell würde man eine Treffsicherheit von 1:(Anzahl Klassen) erwarten
print('Random Ratio')
print(1/num_classes)

# Performance auf dem Test-Set

In [ ]:
# performance vor dem Training
conv_lstm1.eval()
hits = 0
tries = 0
for x_val, y_val in test_loader:
    yhat = conv_lstm1(x_val)
    for target, pred in zip(y_val, yhat):
        tries += 1
        if hit(target, pred):
            hits +=1
            
print('Tries')
print(tries)
print('Number of hits:')
print(hits)
print('Ratio')
print(hits/tries)
# bei einem zufällig gewichteten/untrainierten Modell würde man eine Treffsicherheit von 1:(Anzahl Klassen) erwarten
print('Random Ratio')
print(1/num_classes)